In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferWindowMemory

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ],
    model="gpt-4o-mini"
)

memory = ConversationBufferWindowMemory(
    return_message=True
)

examples = [
    {
    "movie": "Inception",
    "answer": """
🌌🌀🧠
"""
}, 
{
    "movie": "Parasite",
    "answer": """
🏠🌧️🍜
"""
},
{
    "movie": "The Grand Budapest Hotel",
    "answer": """
🏨🎨🕰️
"""
},
{
    "movie": "The Dark Knight",
    "answer": """
🦇🃏🌆
"""
}]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Describe {movie} in three emojis?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    example_prompt,
    ("human", "Describe {movie} in three emojis?"),
])

chain = final_prompt | chat
chain.invoke({
    "movie":"Spirited Away"
})


🌊👧🐉

AIMessageChunk(content='\n🌊👧🐉')